In [ ]:
# Training (Baseline SKLearn) and optional HPO (XGBoost)

In [ ]:
ROLE   = "<YOUR_SAGEMAKER_EXECUTION_ROLE_ARN>"
REGION = "<YOUR_AWS_REGION>"
BUCKET = "<YOUR_S3_BUCKET_NAME>"
PREFIX = "fraud"


In [ ]:
import sagemaker, boto3, os
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.inputs import TrainingInput
from sagemaker.s3 import S3Uploader

sess = sagemaker.Session()
# Upload the code directory to S3 so the training job can access it
s3_code_path = S3Uploader.upload("code", f"s3://{BUCKET}/{PREFIX}/code")
print("Code uploaded to:", s3_code_path)


In [ ]:
# Define and launch training job (Logistic Regression baseline)
est = SKLearn(
    entry_point="train.py",
    source_dir=s3_code_path,
    framework_version="1.2-1",
    role=ROLE,
    instance_type="ml.m5.large",
    instance_count=1,
    output_path=f"s3://{BUCKET}/{PREFIX}/outputs/",
)

channels = {
    "train": TrainingInput(f"s3://{BUCKET}/{PREFIX}/data/processed/train.csv", content_type="text/csv"),
    "val":   TrainingInput(f"s3://{BUCKET}/{PREFIX}/data/processed/val.csv",   content_type="text/csv"),
}

est.fit(channels)
model_artifact = est.model_data
model_artifact


In [ ]:
# (Optional) Hyperparameter tuning: run code from code/hpo.py or paste it here and execute.
# After HPO completes:
# best = tuner.best_estimator()
# best.model_data
